In [1]:
import requests
import json
from bs4 import BeautifulSoup

In [2]:
api_key = 'BUGbVCRM9Mh9ioIDBvYEDQvGfhihYiP7'
start_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'

In [3]:
def get_nyt_url(q, begin_date, end_date):
    url = (start_url+"q={0}&begin_date={1}&end_date={2}&sort=relevance&api-key="+api_key).format(q, begin_date, end_date)
    return url

In [5]:
def get_article(q, begin_date, end_date):
    url = get_nyt_url(q, begin_date, end_date)
    r = requests.get(url)
    #only keep article (not multimedia content)
    articles = [news for news in r.json()['response']['docs'] if news['document_type']=='article']
    return [article for article in articles if article['section_name']!='Sports']

In [4]:
def get_article_urls(q, begin_date, end_date):
    url = get_nyt_url(q, begin_date, end_date)
    r = requests.get(url)
    #only keep article (not multimedia content)
    articles = [news for news in r.json()['response']['docs'] if news['document_type']=='article']
    return [article['web_url'] for article in articles]

In [20]:
get_article('Steve Jobs', 20111001, 20111101)

[{'abstract': 'At a memorial service for her brother, Mona Simpson recalled his love of beauty and his family, and his final moments.',
  'web_url': 'https://www.nytimes.com/2011/10/30/opinion/mona-simpsons-eulogy-for-steve-jobs.html',
  'snippet': 'At a memorial service for her brother, Mona Simpson recalled his love of beauty and his family, and his final moments.',
  'lead_paragraph': 'I grew up as an only child, with a single mother. Because we were poor and because I knew my father had emigrated from Syria, I imagined he looked like Omar Sharif. I hoped he would be rich and kind and would come into our lives (and our not yet furnished apartment) and help us. Later, after I’d met my father, I tried to believe he’d changed his number and left no forwarding address because he was an idealistic revolutionary, plotting a new world for the Arab people.',
  'source': 'The New York Times',
  'multimedia': [],
  'headline': {'main': 'A Sister’s Eulogy for Steve Jobs',
   'kicker': 'Op-Ed C

In [23]:
def get_title_and_text(article_url):
    r = requests.get(article_url)
    soup = BeautifulSoup(r.content)
    article_html_content = soup.find('html').find('body').find('article')
    title = article_html_content.find('header').find('h1').get_text()
    paragraphs = article_html_content.find('section', attrs = {'name':'articleBody'}).find_all('p')
    return [title] + [p.get_text() for p in paragraphs]

In [24]:
corpus_articles = [get_title_and_text(article_url) 
                   for article_url in get_article_urls('Obama', 20081201, 20090101)]

In [25]:
corpus_articles[0]

['Obama Is to Report on Blagojevich Contacts',
 'HONOLULU \x97 President-elect Barack Obama plans Tuesday to release an internal review of the communications that his advisers had with the office of Gov. Rod R. Blagojevich of Illinois, who is accused by federal prosecutors of seeking favors in exchange for filling Mr. Obama’s vacant Senate seat.',
 'Mr. Obama, who is on vacation with his family here in Hawaii, is not planning to answer questions or speak publicly about the report. The review, prepared by lawyers for the Obama transition team, will highlight the telephone calls and contacts that a small circle of Mr. Obama’s advisers had with aides to Mr. Blagojevich last month. It is scheduled to be released Tuesday at 4:30 p.m. Eastern time, aides said.',
 'The report, according to three people familiar with its findings, is a memorandum that will lay out a narrative in about a dozen paragraphs the limited contact Mr. Obama’s advisers had with the governor’s office. Mr. Obama said las